# Naive (Standard) RAG Implementation
### Using FAISS Vector Database and Google Gemini API

This notebook demonstrates a simple three-step RAG process:
1. **Indexing**: Create embeddings and store in FAISS
2. **Retrieval**: Search for relevant documents using similarity
3. **Generation**: Use Gemini to generate contextual responses

In [1]:
# Install Dependencies
!pip install faiss-cpu sentence-transformers google-generativeai numpy

In [2]:
# Import Required Packages
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import json
from typing import List, Tuple

/home/shasshank/.pyenv/versions/ml-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create synthetic knowledge base with 50+ data points
knowledge_base = [
    "Python is a high-level programming language known for its readability and versatility.",
    "Machine Learning is a subset of artificial intelligence that focuses on learning from data.",
    "Deep Learning uses neural networks with multiple layers to process complex patterns.",
    "Natural Language Processing enables computers to understand and process human language.",
    "Computer Vision allows machines to interpret and understand visual information from images.",
    "Data Science combines statistics, programming, and domain expertise to extract insights.",
    "Artificial Intelligence aims to create machines that can perform tasks requiring human intelligence.",
    "Big Data refers to extremely large datasets that require special tools to process.",
    "Cloud Computing delivers computing services over the internet on-demand.",
    "Cybersecurity protects digital systems, networks, and data from cyber threats.",
    "Blockchain is a distributed ledger technology that ensures secure and transparent transactions.",
    "Internet of Things connects everyday devices to the internet for data collection and control.",
    "Quantum Computing uses quantum mechanical phenomena to perform complex calculations.",
    "DevOps combines software development and IT operations to improve collaboration.",
    "Agile methodology emphasizes iterative development and customer collaboration.",
    "APIs (Application Programming Interfaces) allow different software applications to communicate.",
    "Databases store and organize data in a structured way for easy retrieval.",
    "SQL is a programming language designed for managing relational databases.",
    "NoSQL databases are designed to handle unstructured and semi-structured data.",
    "Web development involves creating websites and web applications.",
    "Frontend development focuses on the user interface and user experience.",
    "Backend development handles server-side logic and database interactions.",
    "Mobile app development creates applications for smartphones and tablets.",
    "Version control systems like Git track changes in source code during development.",
    "Software testing ensures that applications work correctly and meet requirements.",
    "Algorithms are step-by-step procedures for solving computational problems.",
    "Data structures organize and store data efficiently in computer memory.",
    "Object-oriented programming organizes code using classes and objects.",
    "Functional programming treats computation as the evaluation of mathematical functions.",
    "Microservices architecture breaks applications into small, independent services.",
    "Containerization packages applications with their dependencies for consistent deployment.",
    "Docker is a popular platform for developing and deploying containerized applications.",
    "Kubernetes orchestrates and manages containerized applications at scale.",
    "REST APIs use HTTP methods to create stateless web services.",
    "GraphQL provides a query language for APIs with flexible data fetching.",
    "Reactive programming handles asynchronous data streams and events.",
    "Design patterns provide reusable solutions to common software design problems.",
    "Code review is the systematic examination of source code to find bugs and improve quality.",
    "Continuous Integration automatically builds and tests code changes.",
    "Continuous Deployment automatically releases code changes to production.",
    "Load balancing distributes incoming requests across multiple servers.",
    "Caching stores frequently accessed data in fast storage for quick retrieval.",
    "Content Delivery Networks distribute content globally for faster access.",
    "Encryption protects data by converting it into a coded format.",
    "Authentication verifies the identity of users accessing a system.",
    "Authorization determines what actions authenticated users can perform.",
    "Data visualization presents information in graphical formats for better understanding.",
    "Business Intelligence uses data analysis to support business decision-making.",
    "ETL processes extract, transform, and load data between different systems.",
    "Data warehouses store large amounts of historical data for analysis.",
    "Real-time analytics processes and analyzes data as it arrives.",
    "Machine Learning models learn patterns from training data to make predictions.",
    "Supervised learning uses labeled data to train predictive models.",
    "Unsupervised learning finds hidden patterns in data without labels.",
    "Reinforcement learning trains agents through interaction with an environment.",
    "Feature engineering creates relevant input variables for machine learning models.",
    "Cross-validation evaluates model performance using different data subsets."
]

print(f"Created knowledge base with {len(knowledge_base)} documents")

Created knowledge base with 57 documents


In [4]:
# Setup Embeddings and FAISS Vector Database
# Initialize sentence transformer for embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all documents
print("Generating embeddings...")
embeddings = embedder.encode(knowledge_base)
embedding_dim = embeddings.shape[1]

# Create FAISS index
index = faiss.IndexFlatIP(embedding_dim)  # Inner Product for cosine similarity
index.add(embeddings.astype('float32'))

print(f"FAISS index created with {index.ntotal} vectors of dimension {embedding_dim}")


Generating embeddings...
FAISS index created with 57 vectors of dimension 384


In [5]:
# Setup Gemini API
API_KEY = ""  # Replace with actual API key
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

In [6]:
# Implement RAG Pipeline
def retrieve_relevant_docs(query: str, top_k: int = 3) -> List[Tuple[str, float]]:
    """Retrieve top-k most relevant documents for the query"""
    # Generate query embedding
    query_embedding = embedder.encode([query])
    
    # Search in FAISS index
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    # Return documents with scores
    results = []
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        if idx < len(knowledge_base):  # Ensure valid index
            results.append((knowledge_base[idx], float(score)))
    
    return results

def generate_response(query: str, context_docs: List[str]) -> str:
    """Generate response using Gemini with retrieved context"""
    # Prepare context
    context = "\n".join([f"- {doc}" for doc in context_docs])
    
    # Create prompt
    prompt = f"""
    Context information:
    {context}
    
    Question: {query}
    
    Please provide a comprehensive answer based on the context information above. 
    If the context doesn't contain relevant information, say so clearly.
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error generating response: {str(e)}"

def naive_rag(query: str, top_k: int = 3) -> dict:
    """Complete Naive RAG pipeline"""
    # Step 1: Retrieval
    relevant_docs = retrieve_relevant_docs(query, top_k)
    
    # Step 2: Augmentation (prepare context)
    context_docs = [doc for doc, score in relevant_docs]
    
    # Step 3: Generation
    response = generate_response(query, context_docs)
    
    return {
        "query": query,
        "retrieved_docs": relevant_docs,
        "response": response
    }


In [7]:
# Cell 7: Test the RAG System
# Test queries
test_queries = [
    "What is machine learning?",
    "How does FAISS work?", 
    "Explain neural networks",
    "What is the difference between SQL and NoSQL?",
    "Tell me about quantum computing"
]

print("Testing Naive RAG System:")
print("=" * 50)

for query in test_queries:
    print(f"\nQuery: {query}")
    result = naive_rag(query)
    
    print(f"\nRetrieved Documents:")
    for i, (doc, score) in enumerate(result["retrieved_docs"], 1):
        print(f"{i}. (Score: {score:.3f}) {doc[:100]}...")
    
    print(f"\nGenerated Response:")
    print(result["response"])
    print("-" * 50)

# Cell 8: Interactive Demo
def interactive_rag():
    """Interactive RAG demo"""
    print("\nInteractive RAG Demo (type 'quit' to exit):")
    
    while True:
        user_query = input("\nEnter your question: ").strip()
        
        if user_query.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
            
        if not user_query:
            continue
            
        result = naive_rag(user_query)
        
        print(f"\nTop 3 Retrieved Documents:")
        for i, (doc, score) in enumerate(result["retrieved_docs"], 1):
            print(f"{i}. {doc}")
        
        print(f"\nResponse: {result['response']}")

Testing Naive RAG System:

Query: What is machine learning?

Retrieved Documents:
1. (Score: 0.822) Machine Learning is a subset of artificial intelligence that focuses on learning from data....
2. (Score: 0.620) Machine Learning models learn patterns from training data to make predictions....
3. (Score: 0.536) Supervised learning uses labeled data to train predictive models....

Generated Response:
Based on the context information provided, machine learning is a subset of artificial intelligence that focuses on learning patterns from data to make predictions. Specifically, it involves training models using data, often labeled data in the case of supervised learning, to recognize patterns and make informed predictions or decisions.

--------------------------------------------------

Query: How does FAISS work?

Retrieved Documents:
1. (Score: 0.237) REST APIs use HTTP methods to create stateless web services....
2. (Score: 0.211) Load balancing distributes incoming requests across mul